### NERC dataset citations - part 1
Code to collect NERC dataset citations from Scholix, CrossRef and DataCite APIs
Produces a csv for each database

In [1]:
import requests, time, json, re, datetime, os, sys
import numpy as np
import pandas as pd
from math import ceil
from datetime import date
from scholix_fun import getNERCDataDOIs, getScholixDatasetCitations, process_citation_results, getPublicationType, countScholixCitations, getCitationString
import crossRef_fun
from crossRef_fun import getDataCiteInfo, getCrossRefCitations, filterCrossRefResults, mergeDFs, getPublicationInfo
from dataCite_fun import getDataCiteCitations_relationTypes, getPublicationInfo_timeCopy
# import exportCitationResultsToCsv
# from Results import convertCSVtoJSON


sys.path.insert(0, '..')


### Get the dataset DOIs

In [ ]:
# this takes approx ~20 mins
# will print output as it goes along to see progress - in JupyterLab right click on the output and select enable scrolling for outputs
dataCite_df = getNERCDataDOIs.getNERCDataDOIs()

## Scholix

### Pass the dataset DOIs to the scholex API to get the citations and their respective DOIs

In [ ]:
# this takes about 8 mins
scholex_df = getScholixDatasetCitations.getScholixDatasetCitations(dataCite_df)

#### Process the citation results

In [ ]:
dataCite_df, scholex_df = process_citation_results.process_citation_results(dataCite_df, scholex_df)

In [ ]:
# filter out gbif registrant code prefix 10.15468
# scholex_df = scholex_df[~scholex_df.pubID.str.contains("10.15468")]
scholex_df = scholex_df[~scholex_df['pubID'].apply(lambda x: str(x)).str.contains("10.15468")]


### Check the DOIs at DOI.org to determine the type of publication and to check there are no duplicates (by preprints etc) 

In [ ]:
# very long 3+ hours
scholex_df = getPublicationType.getPublicationType(scholex_df)

### Output spreadsheet

In [ ]:
today = date.today()

scholex_filename = "Results/Intermediate data/" + 'scholix_citation_publication_info_' + (today.strftime("%d%m%Y")) + '.csv'
scholex_df.to_csv(scholex_filename, index = False)
scholex_df.to_csv("Results/Intermediate data/latest_results_scholix.csv", index = False)

## CrossRef

In [ ]:
# Code for getting results based on a date range

email = "matnic@ceh.ac.uk"
prefix = "10.5285"
start_date = "1990-01-01"
today = date.today()
end_date = today.strftime("%Y-%m-%d")
results_folder_path = "Results/"
results_folder_path_name = results_folder_path + "NERC_EDS_events_from_" + start_date + "_up_to_" + end_date

getCrossRefCitations.getCrossRefCitations_byDates(email, prefix, start_date, end_date, results_folder_path)

In [ ]:
# filter results 
crossRef_df_gbif_filtered2_deduplicated = filterCrossRefResults.filterCrossRefResults(results_folder_path_name)

In [ ]:
# Pass citation info to datacite API to collect relevant info on the datasets, data centres etc
(errors, dataCite_df) = getDataCiteInfo.getDataCiteInfo(crossRef_df_gbif_filtered2_deduplicated)

In [ ]:
(dataset_df, crossRef_df_gbif_filtered2_deduplicated) = mergeDFs.mergeDFs(dataCite_df,crossRef_df_gbif_filtered2_deduplicated)

In [ ]:
crossRef_df_gbif_filtered2_deduplicated = getPublicationInfo.getPublicationInfo(crossRef_df_gbif_filtered2_deduplicated)

### Output spreadsheet

In [ ]:
crossRef_df_processed_filename = "Results/Intermediate data/" + 'dataset_citation_publication_info_' + start_date + "_to_" + end_date + "_retrieved_" + (today.strftime("%d%m%Y")) + '.csv'
crossRef_df_gbif_filtered2_deduplicated.to_csv(crossRef_df_processed_filename, index = False)

with open("Results/Intermediate data/latest_results_crossRef.csv", 'w') as f:
    crossRef_df_gbif_filtered2_deduplicated.to_csv("Results/Intermediate data/latest_results_crossRef.csv",index = False)

## DataCite

In [2]:
relation_type_id_list = ['is-cited-by', 'is-referenced-by', 'is-supplement-to']
dataCite_df_relationTypes = getDataCiteCitations_relationTypes.getDataCiteCitations_relationTypes(relation_type_id_list)

is-cited-by
Total records: 19
Total pages: 1
Status:  200
Page:  1
Final page
is-referenced-by
Total records: 1533
Total pages: 2
Status:  200
Page:  1
https://api.datacite.org/events?page%5Bcursor%5D=MTY3NTk0MDk2OTEyOCw2MDFmMzBjNS04ZTdmLTQ1NTktYWI0Yy0zYjMzODkzZWQ0ZGM&page%5Bsize%5D=1000&prefix=10.5285&relation-type-id=is-referenced-by
Status:  200
Page:  2
Final page
is-supplement-to
Total records: 8
Total pages: 1
Status:  200
Page:  1
Final page
Done!


In [3]:
# remove http from DOI url
doi_list = []
for url in dataCite_df_relationTypes['subj-id']:
    doi = url.replace('https://doi.org/','')
    doi_list.append(doi)
dataCite_df_relationTypes['data_doi'] = doi_list
dataCite_df_relationTypes = dataCite_df_relationTypes.drop(['subj-id'], axis=1)

# doi_list = []
# for url in dataCite_df_relationTypes['obj-id']:
#     doi = url.replace('https://doi.org/','')
#     doi_list.append(doi)
# dataCite_df_relationTypes['pub_doi'] = doi_list # rename to work in getPublicationInfo function 
    
dataCite_df_relationTypes = dataCite_df_relationTypes.rename(columns={"obj-id": "pub_doi_url"})

In [4]:
# drop the rows where the data_doi column value does not start with "10.5285"
dataCite_df_relationTypes = dataCite_df_relationTypes[dataCite_df_relationTypes['data_doi'].str.startswith('10.5285')]

In [6]:
# get dataset metadata 
info_list = []
headers = {'client-id': 'bl.nerc'}
api_url = 'https://api.datacite.org/dois/' 
# for doi in dataCite_df_relationTypes['data_doi']:
for (source_id, relation_type_id, occurred_at, Page_endpoint, data_doi, pub_doi_url) in zip(dataCite_df_relationTypes['source-id'],dataCite_df_relationTypes['relation-type-id'],dataCite_df_relationTypes['occurred-at'],dataCite_df_relationTypes['Page endpoint'],dataCite_df_relationTypes['data_doi'], dataCite_df_relationTypes['pub_doi_url']):
    r = requests.get((api_url + data_doi), headers)
    print(r.status_code, data_doi)
    
    try:
        # process author info
        author_list = []
        for item in r.json()['data']['attributes']['creators']:
            author_list.append(item['name'])

        info_list.append([
            r.json()['data']['attributes']['publisher'],
            data_doi,
            r.json()['data']['attributes']['titles'][0]['title'],
            author_list,
            r.json()['data']['attributes']['publicationYear'],
            r.json()['data']['attributes']['dates'],
            r.json()['data']['attributes']['registered'],
            source_id, relation_type_id, pub_doi_url, occurred_at, Page_endpoint
        ])
    except Exception as e:
        info_list.append(["error",data_doi,"error","error","error","error","error","error","error",pub_doi_url,"error","error"])
        
columns = ['data_publisher', 'data_doi', 'data_title', 'data_authors', 'publicationYear', 'dates', 'registered', 
           'source-id', 'relation-type-id', 'pub_doi_url', 'occurred-at', 'Page endpoint']
dataCite_df = pd.DataFrame(info_list, columns = columns)    
print("Done!")
    

200 10.5285/2641515f-5b76-445c-a936-1da51bf365ad
200 10.5285/6feac38a-5847-46f9-84e4-e7e9d291f935
200 10.5285/db55406b-c9a1-4a9e-88c2-2abbcb4bcad3
200 10.5285/5321bc6e-be35-4ed3-9b56-25598d61ac8f
200 10.5285/c4ecfe25-12f2-453b-ad19-49a19e90ee32
200 10.5285/1f44795b-e596-433c-b69f-caf674880daa
200 10.5285/bc114581-413a-4c62-adb6-556155901f68
200 10.5285/bc114581-413a-4c62-adb6-556155901f68
200 10.5285/bc114581-413a-4c62-adb6-556155901f68
200 10.5285/9ab1541b-e71a-4d75-9907-968d47755e99
200 10.5285/21a8fca5-8eae-48e4-93c0-bc6b4433e34c
200 10.5285/d77dd930-654b-4d09-99bd-8df3b00025a8
200 10.5285/d77dd930-654b-4d09-99bd-8df3b00025a8
200 10.5285/0f074839-1630-4ccd-aa63-84d0da16b28a
200 10.5285/cc1d42de-dfe6-40aa-a1a6-d45cb2fc8293
200 10.5285/dada63fb-c40a-4b13-97ba-c53860881d79
200 10.5285/3952a4fe-683a-42e7-a074-bdec41c8ab16
200 10.5285/3578bae2-4b88-4b2e-93e1-6965dfe1348c
200 10.5285/a1ab8c79-3426-43a4-ab42-6d1b218d1cc6
200 10.5285/0cf552a6-cd62-4da0-8289-8e4bab0a35a8
200 10.5285/65abc40d

In [7]:
# get publication info - takes a long time
dataCite_df_temp = dataCite_df.rename(columns={"pub_doi_url": "subj_id"})
dataCite_df_publication_meta = getPublicationInfo_timeCopy.getPublicationInfo(dataCite_df_temp)

https://doi.org/10.1016/j.scitotenv.2012.05.023 200
https://doi.org/10.1002/2015gl065750 200
https://doi.org/10.17863/cam.20713 200
https://doi.org/10.1111/1365-2656.12728 200
https://doi.org/10.5194/cp-2017-18 200
https://doi.org/10.1002/2016gl068130 200
https://doi.org/10.1029/2007gl032529 200
https://doi.org/10.1029/2009gl040104 200
https://doi.org/10.1029/2009jd012263 200
https://doi.org/10.1002/2015gl065750 200
https://doi.org/10.3189/172756494794587438 200
https://doi.org/10.1029/2007gl032529 200
https://doi.org/10.1029/2009gl040104 200
https://doi.org/10.1111/1365-2656.12798 200
https://doi.org/10.5194/cp-2017-18 200
https://doi.org/10.1029/2018jc013982 200
https://doi.org/10.1175/jcli-d-17-0320.1 200
https://doi.org/10.1111/ele.13129 200
https://doi.org/10.1016/j.jenvrad.2017.06.024 200
https://doi.org/10.1029/2018jc014464 200
https://doi.org/10.5285/4859dc19-e8e9-4148-8c50-cb2ab16dc696 200
https://doi.org/10.5285/65abc40d-e256-414b-8b50-a5569556d1be 200
https://doi.org/10.5285

In [8]:
doi_list = []
for url in dataCite_df_publication_meta['subj_id']:
    doi = url.replace('https://doi.org/','')
    doi_list.append(doi)
dataCite_df_publication_meta['pub_doi'] = doi_list # rename to work in getPublicationInfo function 

In [9]:
#process dataset publisher names
newPublisherLst = []
for dataCentreName in dataCite_df_publication_meta['data_publisher']:
    if type(dataCentreName) == float or dataCentreName is None:
        newPublisherLst.append(dataCentreName)
        continue
    else:
        pass

    dataCentreName_lower = dataCentreName.lower() # make it all lowercase as 'in' operator used below is case sensitive
    if 'polar' in dataCentreName_lower:
        newPublisherLst.append('Polar Data Centre (PDC)')
    elif 'atmospheric' in dataCentreName_lower or 'badc' in dataCentreName_lower or 'earth' in dataCentreName_lower:
        newPublisherLst.append('Centre for Environmental Data Analysis (CEDA)')
    elif 'oceanographic' in dataCentreName_lower:
        newPublisherLst.append('British Oceanographic Data Centre (BODC)')
    elif 'geological' in dataCentreName_lower or 'geoscience' in dataCentreName_lower:
        newPublisherLst.append('National Geoscience Data Centre (NGDC)')
    elif 'environmental information' in dataCentreName_lower:
        newPublisherLst.append('Environmental Information Data Centre (EIDC)')
    elif 'environmental data' in dataCentreName_lower:
        newPublisherLst.append('Centre for Environmental Data Analysis (CEDA)')
    else:
        newPublisherLst.append(dataCentreName)
dataCite_df_publication_meta['publisher_processed'] = newPublisherLst

dataCite_df_publication_meta = dataCite_df_publication_meta.drop(['data_publisher'], axis=1)
dataCite_df_publication_meta = dataCite_df_publication_meta.rename(columns={'publisher_processed':'data_publisher'})

### Output spreadsheet

In [12]:
today = date.today()
dataCite_filename = "Results/Intermediate data/" + 'dataCite_df_events_publication_meta_retrieved_' + (today.strftime("%d%m%Y")) + '.csv'
dataCite_df_publication_meta.to_csv(dataCite_filename, index = False)
dataCite_df_publication_meta.to_csv("Results/Intermediate data/latest_results_dataCite.csv", index = False)
